# Loading data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Import
data = pd.read_csv('dataset_mood_smartphone.csv')

#Convert time column to date time format
data['time']= pd.to_datetime(data['time']) 
data.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00,mood,6.0


# Changing to multi index with time and ID

In [2]:
data = data.set_index(['id', 'time'])
data.head()

Unnamed: 0 variable  value
id      time                                           
AS14.01 2014-02-26 13:00:00           1     mood    6.0
        2014-02-26 15:00:00           2     mood    6.0
        2014-02-26 18:00:00           3     mood    6.0
        2014-02-26 21:00:00           4     mood    7.0
        2014-02-27 09:00:00           5     mood    6.0

# Aggregating the data into multiple time window observations for each patient

We want to reformat the data such that we have a number of observations for each patient, with each observation consisting of some time period e.g one week. Each observation will have a measurement for each attribute (e.g., Avg time spent on weather app over period) with the dependent variable as the average mood over the period. 

In [4]:
#reshaping the dataframe so that each column is a feature, indexed first by patient then time
data2 = pd.pivot_table(data, index=['id', 'time'], columns='variable', values='value')
data2.head()

variable                         activity  appCat.builtin  \
id      time                                                
AS14.01 2014-02-17 12:04:42.394       NaN             NaN   
        2014-02-17 18:28:25.520       NaN             NaN   
        2014-02-18 09:29:51.257       NaN             NaN   
        2014-02-19 14:43:30.575       NaN             NaN   
        2014-02-19 17:29:10.378       NaN             NaN   

variable                         appCat.communication  appCat.entertainment  \
id      time                                                                  
AS14.01 2014-02-17 12:04:42.394                   NaN                   NaN   
        2014-02-17 18:28:25.520                   NaN                   NaN   
        2014-02-18 09:29:51.257                   NaN                   NaN   
        2014-02-19 14:43:30.575                   NaN                   NaN   
        2014-02-19 17:29:10.378                   NaN                   NaN   

variable                         appCat.finance  appCat.game  appCat.office  \
id      time                                                                  
AS14.01 2014-02-17 12:04:42.394             NaN          NaN            NaN   
        2014-02-17 18:28:25.520             NaN          NaN            NaN   
        2014-02-18 09:29:51.257             NaN          NaN            NaN   
        2014-02-19 14:43:30.575             NaN          NaN            NaN   
        2014-02-19 17:29:10.378             NaN          NaN            NaN   

variable                         appCat.other  appCat.social  appCat.travel  \
id      time                                                                  
AS14.01 2014-02-17 12:04:42.394           NaN            NaN            NaN   
        2014-02-17 18:28:25.520           NaN            NaN            NaN   
        2014-02-18 09:29:51.257           NaN            NaN            NaN   
        2014-02-19 14:43:30.575           NaN            NaN            NaN   
        2014-02-19 17:29:10.378           NaN            NaN            NaN   

variable                         appCat.unknown  appCat.utilities  \
id      time                                                        
AS14.01 2014-02-17 12:04:42.394             NaN               NaN   
        2014-02-17 18:28:25.520             NaN               NaN   
        2014-02-18 09:29:51.257             NaN               NaN   
        2014-02-19 14:43:30.575             NaN               NaN   
        2014-02-19 17:29:10.378             NaN               NaN   

variable                         appCat.weather  call  circumplex.arousal  \
id      time                                                                
AS14.01 2014-02-17 12:04:42.394             NaN   1.0                 NaN   
        2014-02-17 18:28:25.520             NaN   1.0                 NaN   
        2014-02-18 09:29:51.257             NaN   1.0                 NaN   
        2014-02-19 14:43:30.575             NaN   1.0                 NaN   
        2014-02-19 17:29:10.378             NaN   1.0                 NaN   

variable                         circumplex.valence  mood  screen  sms  
id      time                                                            
AS14.01 2014-02-17 12:04:42.394                 NaN   NaN     NaN  NaN  
        2014-02-17 18:28:25.520                 NaN   NaN     NaN  NaN  
        2014-02-18 09:29:51.257                 NaN   NaN     NaN  NaN  
        2014-02-19 14:43:30.575                 NaN   NaN     NaN  NaN  
        2014-02-19 17:29:10.378                 NaN   NaN     NaN  NaN

In [5]:
#Replace missing values with 0's - since we intend to take averages 
data2 = data2.fillna(value=0)
data2.head()

variable                         activity  appCat.builtin  \
id      time                                                
AS14.01 2014-02-17 12:04:42.394       0.0             0.0   
        2014-02-17 18:28:25.520       0.0             0.0   
        2014-02-18 09:29:51.257       0.0             0.0   
        2014-02-19 14:43:30.575       0.0             0.0   
        2014-02-19 17:29:10.378       0.0             0.0   

variable                         appCat.communication  appCat.entertainment  \
id      time                                                                  
AS14.01 2014-02-17 12:04:42.394                   0.0                   0.0   
        2014-02-17 18:28:25.520                   0.0                   0.0   
        2014-02-18 09:29:51.257                   0.0                   0.0   
        2014-02-19 14:43:30.575                   0.0                   0.0   
        2014-02-19 17:29:10.378                   0.0                   0.0   

variable                         appCat.finance  appCat.game  appCat.office  \
id      time                                                                  
AS14.01 2014-02-17 12:04:42.394             0.0          0.0            0.0   
        2014-02-17 18:28:25.520             0.0          0.0            0.0   
        2014-02-18 09:29:51.257             0.0          0.0            0.0   
        2014-02-19 14:43:30.575             0.0          0.0            0.0   
        2014-02-19 17:29:10.378             0.0          0.0            0.0   

variable                         appCat.other  appCat.social  appCat.travel  \
id      time                                                                  
AS14.01 2014-02-17 12:04:42.394           0.0            0.0            0.0   
        2014-02-17 18:28:25.520           0.0            0.0            0.0   
        2014-02-18 09:29:51.257           0.0            0.0            0.0   
        2014-02-19 14:43:30.575           0.0            0.0            0.0   
        2014-02-19 17:29:10.378           0.0            0.0            0.0   

variable                         appCat.unknown  appCat.utilities  \
id      time                                                        
AS14.01 2014-02-17 12:04:42.394             0.0               0.0   
        2014-02-17 18:28:25.520             0.0               0.0   
        2014-02-18 09:29:51.257             0.0               0.0   
        2014-02-19 14:43:30.575             0.0               0.0   
        2014-02-19 17:29:10.378             0.0               0.0   

variable                         appCat.weather  call  circumplex.arousal  \
id      time                                                                
AS14.01 2014-02-17 12:04:42.394             0.0   1.0                 0.0   
        2014-02-17 18:28:25.520             0.0   1.0                 0.0   
        2014-02-18 09:29:51.257             0.0   1.0                 0.0   
        2014-02-19 14:43:30.575             0.0   1.0                 0.0   
        2014-02-19 17:29:10.378             0.0   1.0                 0.0   

variable                         circumplex.valence  mood  screen  sms  
id      time                                                            
AS14.01 2014-02-17 12:04:42.394                 0.0   0.0     0.0  0.0  
        2014-02-17 18:28:25.520                 0.0   0.0     0.0  0.0  
        2014-02-18 09:29:51.257                 0.0   0.0     0.0  0.0  
        2014-02-19 14:43:30.575                 0.0   0.0     0.0  0.0  
        2014-02-19 17:29:10.378                 0.0   0.0     0.0  0.0

# Want to obtain an average of all the values for each day - not working as desired

In [8]:
level_values = data2.index.get_level_values
result = (data2.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='1D', level=-1)]).mean())


In [11]:
result

variable                                    activity  appCat.builtin  \
id      time                    time                                   
AS14.01 2014-02-17 12:04:42.394 2014-02-17     0.000           0.000   
        2014-02-17 18:28:25.520 2014-02-17     0.000           0.000   
        2014-02-18 09:29:51.257 2014-02-18     0.000           0.000   
        2014-02-19 14:43:30.575 2014-02-19     0.000           0.000   
        2014-02-19 17:29:10.378 2014-02-19     0.000           0.000   
        2014-02-19 17:42:16.499 2014-02-19     0.000           0.000   
        2014-02-19 17:42:34.292 2014-02-19     0.000           0.000   
        2014-02-19 17:43:18.061 2014-02-19     0.000           0.000   
        2014-02-19 17:44:01.594 2014-02-19     0.000           0.000   
        2014-02-19 18:07:50.055 2014-02-19     0.000           0.000   
        2014-02-19 18:10:06.385 2014-02-19     0.000           0.000   
        2014-02-19 21:25:04.397 2014-02-19     0.000           0.000   
        2014-02-20 10:58:33.924 2014-02-20     0.000           0.000   
        2014-02-20 11:13:44.448 2014-02-20     0.000           0.000   
        2014-02-20 11:16:45.930 2014-02-20     0.000           0.000   
        2014-02-20 13:42:05.309 2014-02-20     0.000           0.000   
        2014-02-20 13:42:56.997 2014-02-20     0.000           0.000   
        2014-02-21 15:36:58.253 2014-02-21     0.000           0.000   
        2014-02-22 17:34:44.325 2014-02-22     0.000           0.000   
        2014-02-22 18:17:55.618 2014-02-22     0.000           0.000   
        2014-02-22 22:32:39.631 2014-02-22     0.000           0.000   
        2014-02-25 11:27:52.973 2014-02-25     0.000           0.000   
        2014-02-25 11:38:26.127 2014-02-25     0.000           0.000   
        2014-02-25 12:29:25.009 2014-02-25     0.000           0.000   
        2014-02-26 13:00:00.000 2014-02-26     0.000           0.000   
        2014-02-26 15:00:00.000 2014-02-26     0.000           0.000   
        2014-02-26 16:19:04.586 2014-02-26     0.000           0.000   
        2014-02-26 18:00:00.000 2014-02-26     0.000           0.000   
        2014-02-26 18:10:37.734 2014-02-26     0.000           0.000   
        2014-02-26 21:00:00.000 2014-02-26     0.000           0.000   
...                                              ...             ...   
AS14.33 2014-05-30 21:58:51.395 2014-05-30     0.000           3.018   
        2014-05-30 21:58:54.421 2014-05-30     0.000           0.000   
        2014-05-30 21:59:57.865 2014-05-30     0.000          21.075   
        2014-05-30 22:00:00.000 2014-05-30     0.375           0.000   
        2014-05-30 22:00:18.943 2014-05-30     0.000          72.213   
        2014-05-30 22:01:31.157 2014-05-30     0.000           1.016   
        2014-05-30 22:01:32.177 2014-05-30     0.000           3.019   
        2014-05-30 22:01:35.197 2014-05-30     0.000          25.076   
        2014-05-30 22:02:00.275 2014-05-30     0.000           0.000   
        2014-05-30 22:02:24.378 2014-05-30     0.000          23.085   
        2014-05-30 22:02:47.464 2014-05-30     0.000          27.386   
        2014-05-30 22:03:14.875 2014-05-30     0.000           4.016   
        2014-05-30 22:03:19.157 2014-05-30     0.000          13.045   
        2014-05-30 22:03:32.207 2014-05-30     0.000           2.004   
        2014-05-30 22:03:34.213 2014-05-30     0.000          47.219   
        2014-05-30 22:04:21.434 2014-05-30     0.000           7.018   
        2014-05-30 22:04:28.453 2014-05-30     0.000           0.000   
        2014-05-30 22:04:31.486 2014-05-30     0.000          22.076   
        2014-05-30 22:04:53.562 2014-05-30     0.000          71.257   
        2014-05-30 22:06:04.821 2014-05-30     0.000           3.005   
        2014-05-30 22:06:07.826 2014-05-30     0.000           0.000   
        2014-05-30 22:07:36.256 2014-05-30     0.000           1.266   
        2014-05-30 22:18:11.591 2014-05-